# Case 10: Backtesting Investment Strategies (US Stocks)

## Importing the Data

In [ ]:
import pandas as pd

In [ ]:
pd.read_csv("dji.csv").info()

In [ ]:
data = pd.read_csv("dji.csv", parse_dates = ["Date"], index_col = "Date")

In [ ]:
data

In [ ]:
data.info()

In [ ]:
df = data.loc["2010-01-01": "2020-03-31", "Close"].to_frame()

In [ ]:
df

## Data Visualization & Returns

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
plt.style.use("seaborn")

In [ ]:
df.describe()

In [ ]:
df.plot(figsize = (20, 12), fontsize = 15)
plt.legend(fontsize = 15)
plt.show()

In [ ]:
df["Return"] = df.pct_change()

In [ ]:
df

In [ ]:
df.dropna(inplace = True)

In [ ]:
df

In [ ]:
df.plot(figsize = (20, 12), secondary_y = "Return", mark_right = True, fontsize = 15)
plt.show()

In [ ]:
df.rename(columns = {"Close":"DJI_Close", "Return":"DJI_Return"}, inplace = True)

In [ ]:
df

## Backtesting a simple Momentum Strategy

__Positions__:

+1: Investing in DJI (__long__ position) <br>
-1: Short Selling DJI (__short__ position)<br>
0: No position (__neutral__)

__Strategies__:

-__Buy and Hold__ (Basic Strategy - passive): Initially Investing into DJI and do nothing (Position: +1 on any given day)

-__Simple Momentum__ (active Strategy to be tested): <br>
__a) Investing (+1)__ into DJI tomorrow if today´s return was __positive__ <br>
__b) Short selling (-1)__ DJI tomorrow if today´s return was __negative__

In [ ]:
df

In [ ]:
df["Position"] = np.sign(df["DJI_Return"])

In [ ]:
df

In [ ]:
df["Strategy_Ret"] = df["Position"].shift() * df["DJI_Return"]

In [ ]:
df

In [ ]:
df.Strategy_Ret.add(1, fill_value = 0).cumprod()

In [ ]:
df["Strategy"] = df.Strategy_Ret.add(1, fill_value = 0).cumprod() * df.iloc[0, 0]

In [ ]:
df

In [ ]:
df[["DJI_Close", "Strategy"]].plot(figsize = (15, 10), fontsize = 15)
plt.title("Simple Momentum Strategy", fontsize = 20)
plt.legend(fontsize = 15)
plt.show()

In [ ]:
def summary_ann(returns):
    summary = returns.agg(["mean", "std"]).T
    summary["Return"] = summary["mean"] * 252
    summary["Risk"] = summary["std"] * np.sqrt(252)
    summary.drop(columns = ["mean", "std"], inplace = True)
    return summary

In [ ]:
summary_ann(df[["DJI_Return", "Strategy_Ret"]])

## Backtesting a simple Contrarian Strategy

__Strategies__:

-__Buy and Hold__ (Basic Strategy): Initially Investing into DJI and do nothing (Position: +1 on all days)

-__Simple Contrarian__ (Strategy to be tested): <br>
__a) Short Selling (-1)__ DJI tomorrow if today´s return was __positive__ <br>
__b) Investing (+1)__ into DJI tomorrow if today´s return was __negative__

In [ ]:
data

In [ ]:
df = data.loc["2010":, "Close"].to_frame()
df["DJI_Return"] = df.Close.pct_change()
df.columns = ["DJI_Close", "DJI_Return"]
df.dropna(inplace = True)
df

In [ ]:
df["Position"] = -np.sign(df["DJI_Return"])

In [ ]:
df

In [ ]:
df["Strategy_Ret"] = df["Position"].shift() * df["DJI_Return"]

In [ ]:
df

In [ ]:
df["Strategy"] = df.Strategy_Ret.add(1, fill_value = 0).cumprod() * df.iloc[0, 0]

In [ ]:
df

In [ ]:
df[["DJI_Close", "Strategy"]].plot(figsize = (15, 10), fontsize = 15)
plt.legend(fontsize = 15)
plt.title("Simple Contrarian Strategy", fontsize = 20)
plt.show()

In [ ]:
summary_ann(df[["DJI_Return", "Strategy_Ret"]])

## More complex Strategies & Backtesting vs. Fitting

In [ ]:
data

In [ ]:
df = data.loc["2010":, "Close"].to_frame()
df["DJI_Return"] = df.Close.pct_change()
df.columns = ["DJI_Close", "DJI_Return"]
df.dropna(inplace = True)
df

In [ ]:
df["Position"] = np.where(df["DJI_Return"] > 0.01, -1, 1)

In [ ]:
df

In [ ]:
df["Strategy_Ret"] = df["Position"].shift() * df["DJI_Return"]

In [ ]:
df

In [ ]:
df["Strategy"] = df.Strategy_Ret.add(1, fill_value = 0).cumprod() * df.iloc[0, 0]

In [ ]:
df

In [ ]:
df[["DJI_Close", "Strategy"]].plot(figsize = (15, 10), fontsize = 15)
plt.legend(fontsize = 15)
plt.title("Simple Contrarian Strategy", fontsize = 20)
plt.show()

In [ ]:
summary_ann(df[["DJI_Return", "Strategy_Ret"]])

Even if a Strategy __seems to outperform__ the basic Strategy, the following issues need to be considered/analyzed as well: <br>
- Backtesting vs. Fitting -> Was the Strategy __fitted__ and optimized on __historical data__? __Forward Testing__ is required!<br>
- __Transaction Costs__ -> Changing Positions trigger Cost. Include Costs!<br>
- __Tax effects__ -> Changing Positions can trigger (earlier) Tax Payments. Include Taxes!

## Simple Moving Averages (Part 1)

In [ ]:
data

In [ ]:
df = data.loc["2010":, "Close"].to_frame()
df["DJI_Return"] = df.Close.pct_change()
df.columns = ["DJI_Close", "DJI_Return"]
df.dropna(inplace = True)
df

In [ ]:
df.DJI_Close.rolling(window = 50).mean()

In [ ]:
df["SMA50"] = df.DJI_Close.rolling(window = 50).mean()

In [ ]:
df[["DJI_Close", "SMA50"]].plot(figsize = (15, 10), fontsize = 15)
plt.legend(fontsize = 15)
plt.show()

In [ ]:
df["SMA200"] = df.DJI_Close.rolling(window = 200).mean()

In [ ]:
df

In [ ]:
df[["SMA50", "SMA200"]].plot(figsize = (15, 10), fontsize = 15)
plt.legend(fontsize = 15)
plt.show()

In [ ]:
df.dropna(inplace= True)

In [ ]:
df

## Simple Moving Averages (Part 2)

__Strategies__:

-__Buy and Hold__ (Basic Strategy): Initially Investing into DJI and do nothing (Position: +1 on all days)

-__SMA Crossover (Momentum)__ (Strategy to be tested): <br>
__a) Investing (+1)__: SMA50 __>__ SMA200 <br>
__b) Short Selling (-1)__: SMA50 __<__ SMA200

In [ ]:
df["Position"] = np.sign(df.SMA50.sub(df.SMA200)) 

In [ ]:
df

In [ ]:
df[["SMA50", "SMA200", "Position"]].plot(figsize = (15, 10), secondary_y = "Position", fontsize = 15)
plt.show()

In [ ]:
df["Strategy_Ret"] = df["Position"].shift() * df["DJI_Return"]

In [ ]:
df

In [ ]:
df["Strategy"] = df.Strategy_Ret.add(1, fill_value = 0).cumprod() * df.iloc[0, 0]

In [ ]:
df

In [ ]:
df[["DJI_Close", "Strategy"]].plot(figsize = (15, 10), fontsize = 15)
plt.legend(fontsize = 15)
plt.title("SMA Strategy", fontsize = 20)
plt.show()

In [ ]:
summary_ann(df[["DJI_Return", "Strategy_Ret"]])

In [ ]:
df["Position"] = -np.sign(df.SMA50.sub(df.SMA200)) 

In [ ]:
df

In [ ]:
df[["SMA50", "SMA200", "Position"]].plot(figsize = (15, 10), secondary_y = "Position", fontsize = 15)
plt.show()

In [ ]:
df["Strategy_Ret"] = df["Position"].shift() * df["DJI_Return"]

In [ ]:
df

In [ ]:
df["Strategy"] = df.Strategy_Ret.add(1, fill_value = 0).cumprod() * df.iloc[0, 0]

In [ ]:
df

In [ ]:
df[["DJI_Close", "Strategy"]].plot(figsize = (15, 10), fontsize = 15)
plt.legend(fontsize = 15)
plt.title("SMA Strategy", fontsize = 20)
plt.show()

In [ ]:
summary_ann(df[["DJI_Return", "Strategy_Ret"]])

## Backtesting the Perfect Strategy (...in case you can predict the future...)

In [ ]:
data

In [ ]:
df = data.loc["2010":, "Close"].to_frame()
df["DJI_Return"] = df.Close.pct_change()
df.columns = ["DJI_Close", "DJI_Return"]
df.dropna(inplace = True)
df

In [ ]:
df["Position"] = np.sign(df["DJI_Return"])

In [ ]:
df

In [ ]:
df["Strategy_Ret"] = df["Position"] * df["DJI_Return"]

In [ ]:
df

In [ ]:
df["Strategy"] = df.Strategy_Ret.add(1, fill_value = 0).cumprod() * df.iloc[0, 0]

In [ ]:
df

In [ ]:
df[["DJI_Close", "Strategy"]].plot(figsize = (15, 10), fontsize = 15, logy = True)
plt.legend(fontsize = 15)
plt.title("The Perfect Strategy", fontsize = 20)
plt.show()

In [ ]:
summary_ann(df[["DJI_Return", "Strategy_Ret"]])